<a href="https://colab.research.google.com/github/dariamm03/Machine_Learning/blob/main/MachineLearning15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание:
Разработать простейший чат бот, QA или KGQA систему.
1) Создать собственный список вопросов и правильных ответов (от 10 вопросов);
2) Создать тестовый набор вопросов для системы (от 1 вопроса, лучше больше)
3) Выполнить предварительную обработку вопросов
4) Преобразовать вопросы в вектора с помощью Bag of Words и TF-IDF
5) Выполнить поиск наиболее схожего вопроса с помощью оценки косинусного расстояния, коэффициента корреляции, расстояния Жаккара, коэффициента сходства Рэнда
6) Вывести ответ исходя из схожести вопроса

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%pip install -U spacy
%pip install -U spacy-lookups-data
!python -m spacy download ru_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.3 MB/s eta 0:00:00
2023-12-15 18:54:42.987962: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 18:54:42.988030: E external/local_xla

In [ ]:
!python -m spacy validate

2023-12-15 18:55:36.939442: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 18:55:36.939500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 18:55:36.940902: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 18:55:36.949752: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 18:55:38.164219: W tensorflow/compiler/tf2

In [ ]:
import spacy
import ru_core_news_lg
nlp = spacy.load("ru_core_news_lg")

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [ ]:
questions = ["Какое из Семи Чудес Света является единственным сохранившимся до наших дней?",
            "Какое животное является символом Австралии?",
            "Какое место на Земле является самым жарким?",
            "Какое место на Земле является самым низким?",
            "Какое животное может летать задом наперед?",
            "Какая еда считается национальным блюдом Мексики?",
            "Какое животное может менять свой цвет, чтобы соответствовать окружающей среде?",
            "Какой памятник является одной из новых семи чудес света?",
            "Какое событие считается началом Нового времени в истории Европы?",
            "Какое событие привело к окончанию Первой Мировой войны?",
            "Какое место в мире считается самым холодным?",
            "Какое животное является самым ядовитым на Земле?"
            ]

answers = ["Пирамиды Гизы",
  "Кенгуру",
  "Долина Смерти в США",
  "Мёртвое море.",
  "Колибри",
  "Тако",
  "Хамелеон",
  "Мачу Пикчу.",
  "Падение Римской империи",
  "Подписание Версальского мирного договора",
  "Антарктида.",
  "Шипохвостый уж"
  ]

In [ ]:
n = len(questions)
parsed = [None] * n
tx2 = [None] * n

for i in range(n):
    parsed[i] = nlp(questions[i])
    tx2[i] = " ".join([token.lemma_ for token in parsed[i] if token.pos_ != 'PUNCT'])
tx2

['какой из семь чудо свет являться единственный сохраниться до наш день',
 'какой животное являться символ австралия',
 'какой место на земля являться самым жаркий',
 'какой место на земля являться самым низкий',
 'какой животное мочь летать задом наперёд',
 'какой еда считаться национальный блюдо мексика',
 'какой животное мочь менять свой цвет чтобы соответствовать окружать среда',
 'какой памятник являться одной из новый семь чудо свет',
 'какой событие считаться начало новый время в история европа',
 'какой событие привести к окончание первый мировой война',
 'какой место в мир считаться самым холодный',
 'какой животное являться самым ядовитый на земля']

In [ ]:
vectorizer = CountVectorizer()
tk = vectorizer.fit_transform(tx2)
tk_tfidf = TfidfTransformer().fit_transform(tk)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Считаем косинусные расстояния между вопросом пользователя и уже заготовленными вопросами
tstat = cosine_similarity(tk, tk)
tstat_tfidf = cosine_similarity(tk_tfidf, tk_tfidf)

In [ ]:
# вопрос, который задаём
question = "Какое время года самое жаркое?"

In [ ]:
# выведем ответ на вопрос, который максимально похож на тот, что введён пользователем
print(question)
max_index = np.argmax(tstat[n-1, 0:(n-2)])
print(answers[max_index])

Какое время года самое жаркое?
Долина Смерти в США


In [ ]:
#Корреляция
tkk = tk.toarray()
tstat_corr = np.corrcoef(tkk)

tkk_tfidf = tk_tfidf.toarray()
tstat_tfidf_corr = np.corrcoef(tkk_tfidf)

In [ ]:
print(question)
max_index = np.argmax(tstat_corr[n-1, 0:(n-2)])
print(answers[max_index])

Какое время года самое жаркое?
Долина Смерти в США


In [ ]:
#Коэффициент сходства Жаккара
jaccard_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        q_tokens_1 = set(tx2[i].split())
        q_tokens_2 = set(tx2[j].split())
        jaccard_similarity=  len(q_tokens_1.intersection(q_tokens_2))/len(q_tokens_1.union(q_tokens_2))
        jaccard_matrix[i, j] = jaccard_similarity

In [ ]:
print(question)
max_index = np.argmax(jaccard_matrix[n-1, 0:(n-2)])
print(answers[max_index])

Какое время года самое жаркое?
Долина Смерти в США


In [ ]:
#Коэффициент сходства Рэнда
from sklearn.metrics import rand_score

rand_matrix = np.zeros((n, n))
rand_tfidf_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        rand = rand_score(tkk[i], tkk[j])
        rand_matrix[i, j] = rand

In [ ]:
print(question)
max_index = np.argsort(rand_matrix[n-1, 0:(n-2)])[-1]
print(1, answers[max_index])

#посмотрим ещё 2 и 3 максимумы
max_2_index = np.argsort(rand_matrix[n-1, 0:(n-2)])[-2]
print(2, answers[max_2_index])

max_3_index = np.argsort(rand_matrix[n-1, 0:(n-2)])[-3]
print(3, answers[max_3_index])


Какое время года самое жаркое?
1 Мёртвое море.
2 Долина Смерти в США
3 Кенгуру


In [ ]:
import warnings

warnings.filterwarnings('ignore')

for i in range(n):
    for j in range(n):
        rand = rand_score(tkk_tfidf[i], tkk_tfidf[j])
        rand_tfidf_matrix[i, j] = rand

warnings.filterwarnings('default')

In [ ]:
print(question)
max_index = np.argmax(rand_tfidf_matrix[n-1, 0:(n-2)])
print(1, answers[max_index])

#посмотрим ещё 2 и 3 максимумы
max_2_index = np.argsort(rand_tfidf_matrix[n-1, 0:(n-2)])[-2]
print(2, answers[max_2_index])

max_3_index = np.argsort(rand_tfidf_matrix[n-1, 0:(n-2)])[-3]
print(3, answers[max_3_index])

Какое время года самое жаркое?
1 Долина Смерти в США
2 Долина Смерти в США
3 Кенгуру
